In [61]:
from bs4 import BeautifulSoup
import requests
import re
import json
import pandas as pd

In [62]:
url = "https://www.millenaire3.com/ressources/2023/changer-les-comportements-revue-et-enseignements-des-actions-metropolitaines"
url2 = "https://www.millenaire3.com/ressources/2023/agir-sur-les-modes-de-vie-une-nouvelle-grille-d-analyse"
url3 = "https://www.millenaire3.com/dossiers/2022/resilience-quelles-articulations-entre-acteurs-publics-et-initiatives-du-territoire"
url4 = "https://www.millenaire3.com/dossiers/2022/resilience-quelles-articulations-entre-acteurs-publics-et-initiatives-du-territoire/institut-transitions-notre-but-c-est-de-former-ceux-qui-peuvent-agir-des-aujourd-hui"
url5 = "https://www.millenaire3.com/dossiers/2022/resilience-quelles-articulations-entre-acteurs-publics-et-initiatives-du-territoire/les-alchimistes-lyon-alimentation-gaspillage-compostage-aucun-de-ces-sujets-ne-laisse-indifferent"
url6 = "https://www.millenaire3.com/ressources/2023/crise-ecologique-le-metabolisme-de-l-economie-au-caeur-des-enjeux"
url7 = "https://www.millenaire3.com/dossiers/2015/la-soierie-lyonnaise-entre-tradition-et-innovation-archives/la-soie-a-lyon-une-initiative-du-pouvoir-royal"
urls = [url, url2, url3, url4, url5, url6, url7]

with open("./M3urls.txt", "r") as f:
        urls = f.read().split("\n")

In [70]:
def scrap_page(url, container, mult_items, errors):
    
    html = requests.get(url)
    if(html.status_code) != 200:
        print(url, html.status_code)
        errors["deadlink"].append(url)
    soup = BeautifulSoup(html.text, "html.parser")

    mult_items[url] = {"auteurs" : [], "tags" : [], "liens" : []}

    #TYPE
    type_liste = soup.find_all("p", {"class" : re.compile("^over-under")})
    for typ in type_liste:
        if re.search("Étude|Dossier|Interview|Article", typ.text) == None:
            pass
        else:
            if re.search("Interview", typ.text) == None:
                type = typ.text.replace("\n", "").replace("\t", "").replace(" ", "")
            else:
                type = "Interview"
                itw_de = typ.text.replace("\n", "").replace("\t", "")
    if len(type) > 0:
        container["TYPE"].append(type)
    else:
        errors["TYPE"].append(url)
        container["TYPE"].append("NA")

    #RESUME
    if type == "Dossier":
        resume = soup.find("div", {"class" : "eztext-field"})
    elif type == "Interview":
        resume = soup.find("div", {"class" : "ezrichtext-field"})
    else: 
        resume = soup.find("div", {"class" : "eztext-field"})
    if len(resume) > 0:
        container["RESUME"].append(resume.text)
    else:
        errors["RESUME"].append(url)
        container["RESUME"].append("NA")

    #ARTICLE
    if type != "Interview":
        article = soup.find("div",  {"class" : "ezrichtext-field"}).text
    else:
        lines = soup.find_all("div", {"class" : "interview-item"})
        article = ""
        for i, line in enumerate(lines):
            article += line.find("p", {"class" : "over-under-line"}).text + " "
            temp = line.find_all("div", {"class" : "ezrichtext-field"})
            for info in temp:
                article += info.find("p").text
    if len(article) > 0:
        container["ARTICLE"].append(article)
    else:
        errors["ARTICLE"].append(url)
        container["ARTICLE"].append("NA")

    #TITRE
    titre = soup.find("h1", {"class": "uppercase h1-special"}).contents[0]
    titre = titre.replace("\n", "").replace("\t", "").strip()
    if len(titre) > 0:
        container["TITRE"].append(titre)
    else:
        errors["TITRE"].append(url)
        container["TITRE"].append("NA")
    
    #ID - in db
    try:
        ID = soup.find("article", {"class" : "sidebar-block-classic sidebar-block-download"})
        ID = ID.a["href"].split("/")
        ID = ID[len(ID)-1]
        container["IDinDB"].append(ID)
    except:
        container["IDinDB"].append("NA")

    #DATE
    date = soup.find("span", {"class" : "publication_date"}).text
    if len(date) > 0:
        container["DATE"].append(date)
    else:
        errors["DATE"].append(url)
        container["DATE"].append("NA")

    #TAGS
    tags_liste = soup.find_all("ul", {"class": "list-inline keywords"})
    tags = []
    for tag in tags_liste:
        a = tag.find_all("a")
        for tag in a:
            if re.search("tag", tag["href"]) != None:
                tags += tag.contents
    if len(tags) > 0:
        mult_items[url]["tags"] = tags
    else:
        mult_items[url]["tags"] = "NA"

    #AUTHORS
    if type == "Interview":
        liste = soup.find_all("ul", {"class": "list-inline keywords"})
        authors = []
        for elem in liste:
            a = elem.find_all("a")
            for author in a:
                if re.search("auteurs", author["href"]) != None:
                    authors.append(author.contents[0].strip())
    else:
        auth = soup.find_all("span", {"class" : "author-name"})
        authors = []
        for author in auth:
            author = repr(author.text).replace("\\xa0", " ").replace("\\t", "").replace("\\n", "").replace("'", "")
            if author not in authors:
                authors.append(author)
    if len(authors) > 0:
        mult_items[url]["auteurs"] = authors
    else:
        mult_items[url]["auteurs"] = "NA"
        
    #Liens
    page_links = soup.find_all("h3", {"class" : "h3 uppercase"})
    links = []
    for link in page_links:
        links.append("https://www.millenaire3.com"+link.a["href"])
    if len(links) > 0:
        mult_items[url]["liens"] = links
    else:
        mult_items[url]["liens"] = "NA"

    #print(url)
    container["URL"].append(url)

    return(container, mult_items, errors)


In [71]:
container = {"URL" : [], 
             "TITRE" : [], 
             "IDinDB" : [], 
             "TYPE" : [], 
             "RESUME" : [], 
             "ARTICLE" : [],
             "DATE" : []}

mult_items = {}

errors = {
    "TITRE" : [],
    "TYPE" : [],
    "RESUME" : [],
    "ARTICLE" : [],
    "DATE" : [],
    "deadlink" : []
}

for url in urls[6:20]:
    try:
        scrap_page(url, container, mult_items, errors)
    except:
        print("Not an article")

https://www.millenaire3.com/generated_pdf/12280 404
Not an article
Not an article
Not an article
https://www.millenaire3.com/dossiers/L-agenda-sante-archives/L-agenda-sante-n-5 404
Not an article
Not an article


In [72]:
for keys in container:
    print(keys, ":", len(container[keys]))

URL : 9
TITRE : 9
IDinDB : 9
TYPE : 9
RESUME : 9
ARTICLE : 9
DATE : 9


In [73]:
pd.DataFrame(container)

,URL,TITRE,IDinDB,TYPE,RESUME,ARTICLE,DATE
0,https://www.millenaire3.com/dossiers/2020/covi...,Covid-19 / Relectures de corpus : Tous aux arc...,NA,Dossier,Malgré une somme considérable d’études qui pré...,Découvrez les documents ici : ■ Santé publique...,01/09/2020
1,https://www.millenaire3.com/Interview/2021/joe...,Joël de Rosnay : « “Résilience” et “sagesse” s...,NA,Interview,"Complexité, diversité et mélange des parcours,...",Avec la publication en 1975 de votre ouvrage «...,03/06/2021
2,https://www.millenaire3.com/Interview/2019/ent...,"Entrepreneur innovant, social et solidaire : l...",NA,Interview,"Créé fin 2012, le Groupement Régional Alimenta...",Comment en êtes-vous venu à l’entrepreneuriat ...,05/12/2018
3,https://www.millenaire3.com/dossiers/2015/attr...,Développer l'économie de proximité -3. Le ress...,12827,Étude,Cette étude a été initiée dans le cadre de la ...,Ce troisième cahier explore le ressort de la c...,01/11/2013
4,https://www.millenaire3.com/dossiers/2020/veil...,Veille M3 / Les nouvelles pratiques sportives ...,NA,Article,"Fièvre statistique, avalanches de nombres, omn...",Des pratiques individuelles au sport professio...,14/06/2021
5,https://www.millenaire3.com/Interview/2004/les...,Les enjeux pour un laboratoire en santé animal...,NA,Interview,Merial est le premier groupe mondial spécialis...,NA,04/03/2004
6,https://www.millenaire3.com/Interview/2020/bar...,Barbara Jacques : recruter des compétences dan...,NA,Interview,La Direction de la Prospective et du Dialogue ...,Quelle place occupe selon vous la question des...,21/04/2020
7,https://www.millenaire3.com/dossiers/Covid-19-...,Covid-19 / Relectures de corpus : Tous aux arc...,NA,Dossier,Malgré une somme considérable d’études qui pré...,Découvrez les documents ici : ■ Santé publique...,01/09/2020
8,https://www.millenaire3.com/dossiers/2021/econ...,L'économie circulaire,218211,Étude,L’économie circulaire vise à changer de paradi...,Sommaire : Décryptage● Des racines anciennes ...,07/09/2017


In [64]:
print(json.dumps(container, indent = 3, ensure_ascii=False))
print(json.dumps(mult_items, indent = 3, ensure_ascii=False))

{
   "URL": [
      "https://www.millenaire3.com/dossiers/2020/covid-19-relectures-de-corpus-tous-aux-archives",
      "https://www.millenaire3.com/Interview/2021/joel-de-rosnay-resilience-et-sagesse-sont-les-mots-cles-pour-pouvoir-envisager-le-futur",
      "https://www.millenaire3.com/Interview/2019/entrepreneur-innovant-social-et-solidaire-l-exemple-du-grap",
      "https://www.millenaire3.com/dossiers/2015/attractivite-de-la-metropole-et-societe-du-temps-libre-archives/Developper-l-economie-de-proximite-3.-Le-ressort-de-la-captation-de-revenus",
      "https://www.millenaire3.com/Interview/2004/les-enjeux-pour-un-laboratoire-en-sante-animale-comme-merial",
      "https://www.millenaire3.com/Interview/2020/barbara-jacques-recruter-des-competences-dans-les-industries-de-la-sante",
      "https://www.millenaire3.com/dossiers/Covid-19-Relectures-de-corpus-Tous-aux-archives",
      "https://www.millenaire3.com/dossiers/2021/economie-circulaire-au-dela-du-recyclage-comment-transformer-l-

In [143]:
html = requests.get("https://www.millenaire3.com/Interview/2004/les-enjeux-pour-un-laboratoire-en-sante-animale-comme-merial")
soup = BeautifulSoup(html.text, "html.parser")
resume = soup.find("div", {"class" : "ezrichtext-field"}).text
article = soup.find_all("div",  {"class" : "ezrichtext-field"})
for i, art in enumerate(article):
    if art.text != resume:
        p = art.find_all("p")
        for para in p:
            tmp = para.contents
            for var in tmp:
                print(re.sub("<.*>", "", var))

TypeError: expected string or bytes-like object